In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import time
import datetime as dt
import os
import seaborn as sns
import scipy.stats as sts

import random
import string

In [ ]:
from nb_vars import FOLDER_OUTPUT_BASE, CUTOFF_SPIKES_HZ

## Load df of filtered peaks

In [ ]:
CONDITION_NAME = "pulse_duration"
CONDITION = "200"

folder_input =  f'{FOLDER_OUTPUT_BASE}/{CONDITION_NAME}_{CONDITION}'

In [ ]:
df_peaks = pd.read_csv(f'{folder_input}/df_peaks_full_{CONDITION_NAME}_{CONDITION}_freq_{CUTOFF_SPIKES_HZ}.csv')

In [ ]:
df_peaks

## Analysis of ISIs

In [ ]:
df_ISIs = pd.DataFrame(columns=df_peaks.columns)
list_wells = df_peaks['well'].drop_duplicates().values 

for well in list_wells:
    df_well = df_peaks[df_peaks['well'] == well]
    list_electrodes = df_well['electrode'].drop_duplicates().values 

    for electrode in list_electrodes:
        df_well_electrode = df_well[df_well['electrode'] == electrode].copy()
        diff_times = df_well_electrode['time'].iloc[1:].values - df_well_electrode['time'].iloc[:-1].values

        df_well_electrode.loc[df_well_electrode.index[:-1], 'time'] = diff_times

        df_ISIs = pd.concat([df_ISIs, df_well_electrode.iloc[:-1]])

df_ISIs['cond-treat'] = df_ISIs['condition'] + '-' + df_ISIs['treatment']

In [ ]:
def kde_plot(df_ISIs, x_col, g_col, log=True):
    sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

    # Create the data
    df = pd.DataFrame(dict(x=df_ISIs[x_col].values, g=df_ISIs[g_col].values))
    df = df[df['x'] > 0]

    # Initialize the FacetGrid object
    pal = sns.cubehelix_palette(len(df_ISIs[g_col].drop_duplicates().values), rot=-.25, light=.7)
    g = sns.FacetGrid(df, row="g", hue="g", aspect=15, height=.5, palette=pal)

    # Draw the densities in a few steps
    g.map(sns.kdeplot, "x", log_scale=log, 
        bw_adjust=.5, clip_on=False,
        fill=True, alpha=1, linewidth=1.5)
    g.map(sns.kdeplot, "x", clip_on=False, color="w", lw=2, bw_adjust=.5, log_scale=log,)

    # passing color=None to refline() uses the hue mapping
    g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)


    # Define and use a simple function to label the plot in axes coordinates
    def label(x, color, label):
        ax = plt.gca()
        ax.text(0, .2, label, fontweight="bold", color=color,
                ha="left", va="center", transform=ax.transAxes)


    g.map(label, "x")

    # Set the subplots to overlap
    g.figure.subplots_adjust(hspace=-.25)

    # Remove axes details that don't play well with overlap
    g.set_titles("")
    g.set(yticks=[], ylabel="")
    g.despine(bottom=True, left=True)

In [ ]:
kde_plot(df_ISIs, x_col='time', g_col='well', log=True)

In [ ]:
kde_plot(df_ISIs, x_col='time', g_col='cond-treat', log=True)

In [ ]:
kde_plot(df_ISIs[(df_ISIs['time'] > 0.3) & (df_ISIs['time'] < 5)], x_col='time', g_col='well', log=False)

In [ ]:
kde_plot(df_ISIs[(df_ISIs['time'] > 0.5) & (df_ISIs['time'] < 5)], 
         x_col='time', g_col='cond-treat', log=False)